In [1]:
import pandas as pd

In [16]:
raw_data = pd.read_csv("Raw_data/1_rawdata.csv")
groups = pd.read_csv("Raw_data/2_groups.csv")

### Dataset cleaning
###### For the analysis of user behavior, only morning and evening brushing sessions are considered. If multiple brushing events occur within a short interval—specifically, if they are separated by less than two minutes—they are treated as a single session. Any session lasting less than 20 seconds is considered a random error and excluded from the analysis. In cases where a user brushes multiple times in either the morning or evening, only the longest session is retained, and all shorter sessions are discarded.

In [17]:
# dataset stucture
raw_data.head(2)

,ID,TimestampUTC,UpTime,DownTime,LeftTime,RightTime,NoneTime,Unnamed: 7
0,PB2500017115,Mon Sep 18 2017 08:03:18 GMT+0100 (BST),6.6,0.1,0.3,0.3,5.7,NaN
1,PB2500017115,Mon Sep 18 2017 08:06:32 GMT+0100 (BST),0.0,3.8,1.8,0.0,1.4,NaN


In [20]:
# Timestamp cleaning and reloading

raw_data = raw_data[~pd.isna(raw_data.ID)]
raw_data['TimestampUTC'] = pd.to_datetime(raw_data['TimestampUTC'])


In [21]:
# Extracting morning and evening sessions

raw_data['hour'] = raw_data['TimestampUTC'].dt.hour
time_of_day = lambda row: 'evening' if row.hour > 14 else 'morning'
raw_data['sessions'] = raw_data.apply(time_of_day, axis = 1)



In [23]:
# Merge brush sessions that are less than 2 minutes apart into a single brush session
raw_data = raw_data.sort_values(['ID', 'TimestampUTC'])
raw_data['one_session_flag'] = ((raw_data.ID == raw_data.ID.shift())
                            & ( raw_data.sessions == raw_data.sessions.shift())
                            & ((raw_data.TimestampUTC - raw_data.TimestampUTC.shift()).astype('timedelta64[m]') < 2))

raw_data['session_id'] = (raw_data.one_session_flag == False).cumsum()
        
        

In [25]:
# The total length of a brush session is the sum of the up, down, left, right and none times.

time_brush = raw_data[['UpTime', 'DownTime', 'LeftTime', 'RightTime', 'NoneTime']]

raw_data['sub_total_time'] = time_brush.sum(axis =1)

raw_data = raw_data.merge(raw_data.groupby(['ID', 'session_id']).sub_total_time.sum().rename('total_time')
                        , on = ['ID', 'session_id'])

In [26]:
#  Discard brush sessions that are less than 20 seconds in total
raw_data = raw_data[raw_data.total_time >= 20]

In [29]:
#  The longest sesstions

raw_data['day'] = raw_data.TimestampUTC.dt.date

dataset = (raw_data.merge(raw_data.groupby(['ID', 'day', 'sessions']).total_time.max()
                        , on = ['ID', 'day', 'sessions', 'total_time'], how = 'inner')
                           [['ID','day','sessions', 'total_time']]
                        ).drop_duplicates()


### User Information
#### We need to understand general user brushing behavior over the course of a week. To support this, we’ve defined a set of key questions that will help quantify user engagement and identify patterns in morning and evening usage.
#### How frequently do users brush in the morning and in the evening throughout the week?
##### Metric: Number of brushing events per user for each day of the week
#### On how many days did a user brush both in the morning and evening?
##### Metric: Count of days in which both a valid morning and evening session were recorded.
#### What is the total number of valid brushing sessions per user during the week?
#####  Metric: Sum of all valid morning and evening sessions across the week, excluding short or erroneous sessions.
#### How much time does a user spend brushing on average per session?
#####  Metric: Average session duration calculated using only valid brushing sessions.

In [31]:
# User information

#How frequently do users brush in the morning and in the evening throughout the week?

dataset['day'] = pd.to_datetime(dataset['day'])
dataset['dow'] = dataset['day'].dt.day_name()

num_of_session_dow = (pd.DataFrame(dataset.groupby(['ID','dow'])
                        .sessions.count().rename('num_sessions')).reset_index())

#On how many days did a user brush both in the morning and evening?

num_twice_day = (num_of_session_dow[num_of_session_dow.num_sessions >1].
                groupby('ID').dow.count().rename('total-brushes'))


# What is the total number of valid brushing sessions per user during the week?

dataset['week'] = dataset['day'].dt.week
num_week = dataset.groupby(['ID','week']).sessions.count().rename('twice-brushes')


# How much time does a user spend brushing on average per session?
avr_time_session = dataset.groupby(['ID','week']).total_time.mean().rename('vg-brush-time')

/var/folders/9t/5gcr47m91839pxy0fm6mhjw00000gn/T/ipykernel_96089/195175636.py:19: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  dataset['week'] = dataset['day'].dt.week


In [40]:
final_table = (num_of_session_dow.pivot(index='ID', 
                        columns='dow', values='num_sessions').reset_index())

final_table = final_table.merge(groups, left_on = 'ID', right_on = 'PBID')

final_table = (final_table.merge(num_twice_day, on = 'ID')
                        .merge(num_week, on = 'ID' )
                        .merge(avr_time_session, on = 'ID')
                        .rename(columns = {'Monday': 'mon'
                                          , 'Tuesday' : 'tue'
                                          , 'Wednesday' : 'wed'
                                          , 'Thursday' : 'thu'
                                          , 'Friday' : 'fri'
                                          , 'Saturday': 'sat'
                                          , 'Sunday': 'sun'}).fillna(0))

final_table = final_table[['group', 'PBID', 'mon', 'tue', 'wed', 'thu', 'fri'
                          , 'sat', 'sun', 'total-brushes', 'twice-brushes', 'vg-brush-time']]
final_table.to_csv('Outcome/individual_metrics.csv')

###  Group Dynamics 
#### To evaluate brushing behavior across different user groups, we aim to analyze group-level engagement and performance. This will help us identify which groups demonstrate stronger brushing habits and where improvements may be needed.

#### How active are users in each group overall?
##### Metric: Total number of valid brushing sessions recorded per group.

#### How consistently do users in each group brush?
##### Metric: Average number of valid brushing sessions per user within each group.

#### How much time do users in each group spend brushing?
##### Metric: Average brushing duration per user, considering only valid sessions.

#### Which group shows the highest overall brushing engagement and quality?
##### Metric: A composite performance indicator (e.g., highest total sessions, highest average duration, or highest percentage of users brushing twice daily), used to identify the top-performing group.

In [36]:


# How active are users in each group overall?

dataset2 = dataset.merge(groups, left_on = 'ID', right_on = 'PBID')
total_sessions = pd.DataFrame(dataset2.groupby('group').sessions.count().rename('total_sessions'))

# How consistently do users in each group brush?

temp_count = pd.DataFrame(dataset2.groupby(['group', 'PBID']).sessions.count().rename('avg_sessions_per_user'))
user_sessions = temp_count.groupby('group').avg_sessions_per_user.mean()

# How much time do users in each group spend brushing?

temp_long = pd.DataFrame(dataset2.groupby(['group', 'PBID']).total_time.mean().rename('avg_time_per_user'))
user_long = temp_long.groupby('group').avg_time_per_user.mean()

In [37]:
# Merge all session together for ranking
group_data = (total_sessions.merge(user_sessions, on = 'group')
                            .merge(user_long, on = 'group')).reset_index()

In [41]:
# Average perfomance and ranking
group_data['performance'] =( (group_data.avg_time_per_user * group_data.avg_sessions_per_user)
                        / group_data.total_sessions)

group_data = group_data.sort_values('performance', ascending= False)

group_data['rank'] = group_data.performance.rank( axis = 0, ascending = False)
group_data.to_csv('Outcome/group_metrics.csv')